# Практическая работа №4

## Импорт библиотек

In [9]:
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## Подготовка датасета

В видео к практической работе был представлен более сложный и времязатратный способ формирования датафрейма. Я не стал множить сущности и воспользовался вариантом ниже, т.к. формировать датафреймы вручную и писать циклы умею. Если Вы сочтете, что я "читер", то могу обработать json циклами и сформировать таблицу также как это было показано в видео :)

In [ ]:
# Открываем json
with open('parsed.json', 'r', encoding='utf8') as file:
    data = json.load(file)

In [ ]:
# Превращаем его в DataFrame
df = pd.DataFrame(data['data'])
df

In [ ]:
# Подкорректируем названия колонок
df.columns = ['title', 'work_experience', 'salary', 'region']

In [ ]:
# Запишем датасет в csv-таблицу
df.to_csv('parsed_hh.csv', index=False)

## Исследовательский анализ, подготовка и предобработка данных

In [333]:
# Откроем наш датасет из уже записанного csv-файла
df = pd.read_csv('parsed_hh.csv')
df

,title,work_experience,salary,region
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,Астана
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,Нур-Султан (Астана)
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,Алматы
4,Backend разработчик,1–3 года,з/п не указана,Астана
...,...,...,...,...
4279,Специалист службы поддержки с техническими зна...,не требуется,от 15 000 до 40 000 руб. на руки,Ижевск
4280,Web-разработчик junior,не требуется,до 70 000 руб. на руки,Санкт-Петербург
4281,Web-разработчик,не требуется,з/п не указана,Нижний Новгород
4282,Младший аналитик (IT),не требуется,от 100 000 до 130 000 руб. на руки,Москва


In [334]:
# Выведем общую информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4284 entries, 0 to 4283
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            4283 non-null   object
 1   work_experience  4284 non-null   object
 2   salary           4284 non-null   object
 3   region           4284 non-null   object
dtypes: object(4)
memory usage: 67.0+ KB


Имеем 4 столбца и 4284 строки. Пропуски в датасете отсутствуют.  
Сделам проверку на полные дубликаты данных:

In [335]:
df.duplicated().sum()

0

Дубликаты отсутствуют.

Далее значения из столбцов `salary` и `work_experience` переведем в числовой формат.  
Так как многие зарплаты и опыт указаны в виде диапазона, то добавим дополнительные стобцы в датасет (пока с пустыми ячейками), в которые в дальнейшем запишем минимальное значение из диапазона и максимальное значение.

In [336]:
# Добавляем новые столбцы
df['salary_min'] = np.nan
df['salary_max'] = np.nan
df['exp_min'] = np.nan
df['exp_max'] = np.nan

Для того, чтобы выделить числовые значения из текста напишем две функции для одной строки, которые позже применим к датасету методом `apply()`. 

- `salary_to_num` - переведет данные о зарплате в числа;
- `exp_to_num` - переведет данные о требуемом опыте в числа.

In [337]:
def salary_to_num(row):
    number_1 = []
    number_2 = []
    last_symbol = 'start'
    m = ''
    val = row['salary']
    salary = str(row['salary']).replace(' ', '')
    salary = list(''.join(salary))
    salary_min = row['salary_min']
    salary_max = row['salary_max']
    
    for i in salary:
        if m.isnumeric() == True and i.isnumeric() == False:
            last_symbol = 'next'
        if last_symbol == 'start':
            if i.isnumeric() == True:
                number_1.append(i)
        elif last_symbol == 'next':
            if i.isnumeric() == True:
                number_2.append(i)
        m = i
    
    try:
        number_1 = int(''.join(number_1))
    except:
        number_1 = 0
    try:
        number_2 = int(''.join(number_2))
    except:
        number_2 = 0
    
    if number_1 != 0 and number_2 != 0:
        row.loc['salary_min'] = number_1
        row.loc['salary_max'] = number_2
    elif number_1 != 0 and number_2 == 0:
        row.loc['salary_min'] = 0
        row.loc['salary_max'] = number_1
    else:
        row.loc['salary_min'] = 0
        row.loc['salary_max'] = 0
        
    row.loc['salary'] = val
    
    return row


In [338]:
def exp_to_num(row):
    number_1 = []
    number_2 = []
    last_symbol = 'start'
    m = ''
    val = row['work_experience']
    exp = str(row['work_experience']).replace(' ', '')
    exp = list(''.join(exp))
    exp_min = row['exp_min']
    exp_max = row['exp_max']
    
    for i in exp:
        if m.isnumeric() == True and i.isnumeric() == False:
            last_symbol = 'next'
        if last_symbol == 'start':
            if i.isnumeric() == True:
                number_1.append(i)
        elif last_symbol == 'next':
            if i.isnumeric() == True:
                number_2.append(i)
        m = i
    
    try:
        number_1 = int(''.join(number_1))
    except:
        number_1 = 0
    try:
        number_2 = int(''.join(number_2))
    except:
        number_2 = 0
    
    if (number_1 != 0 and number_2 != 0) or (number_1 != '' and number_2 != ''):
        row.loc['exp_min'] = number_1
        row.loc['exp_max'] = number_2
    elif (number_1 != 0 and number_2 == 0) or (number_1 != '' and number_2 == ''):
        row.loc['exp_min'] = 0
        row.loc['exp_max'] = number_1
    else:
        row.loc['exp_min'] = 0
        row.loc['exp_max'] = 0
        
    row.loc['work_experience'] = val
    
    return row

In [339]:
# Применим функции к датасету
df = df.apply(salary_to_num, axis=1)
df = df.apply(exp_to_num, axis=1)

In [340]:
df

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана,200000,1500000,1,3
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,Астана,4500,5000,3,6
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,Нур-Султан (Астана),150000,200000,0,0
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,Алматы,2000,4000,3,6
4,Backend разработчик,1–3 года,з/п не указана,Астана,0,0,1,3
...,...,...,...,...,...,...,...,...
4279,Специалист службы поддержки с техническими зна...,не требуется,от 15 000 до 40 000 руб. на руки,Ижевск,15000,40000,0,0
4280,Web-разработчик junior,не требуется,до 70 000 руб. на руки,Санкт-Петербург,0,70000,0,0
4281,Web-разработчик,не требуется,з/п не указана,Нижний Новгород,0,0,0,0
4282,Младший аналитик (IT),не требуется,от 100 000 до 130 000 руб. на руки,Москва,100000,130000,0,0


Проверим датасет на пропуски

In [341]:
df.isna().sum()

title              1
work_experience    0
salary             0
region             0
salary_min         0
salary_max         0
exp_min            0
exp_max            0
dtype: int64

Есть один пропуск. Выведем строку с ним на экран

In [342]:
df[df['title'].isna()]

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
47,NaN,undefinite,undefinite,undefinite,0,0,0,0


Данные с пропуском не имеют ценности, поэтому удалим их.

In [343]:
df.dropna(inplace=True)

In [344]:
df.isna().sum()

title              0
work_experience    0
salary             0
region             0
salary_min         0
salary_max         0
exp_min            0
exp_max            0
dtype: int64

Проверим строки, в которых может встречаться значение 'undefinite'.

In [345]:
columns_list = df.columns
wrong_list = []
for column in columns_list:
    if len(df[df[column]=='undefinite']) > 0:
        for i in df[df[column]=='undefinite'].index:
            if i not in wrong_list:
                wrong_list.append(i)
wrong_list

[235, 742, 1333, 1694, 3285, 3294, 3525]

Таких строк 7. Выведем их на экран.

In [346]:
df.loc[wrong_list]

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
235,undefinite,undefinite,RocketData,Минск,0,0,0,0
742,undefinite,undefinite,Ventra,Москва,0,0,0,0
1333,undefinite,undefinite,Leads,Москва,0,0,0,0
1694,undefinite,undefinite,ООО Защищенные Телекоммуникации,Тула,0,0,0,0
3285,undefinite,undefinite,ООО Лаборатория Наносемантика,Москва,0,0,0,0
3294,undefinite,undefinite,КОНТРОЛ лизинг,Москва,0,0,0,0
3525,undefinite,undefinite,Группа компаний «БПЦ»,Москва,0,0,0,0


Данные были собраны неверно. Удалим их, т.к. восстановить информацию возможности нет, данных мало и от них не будет пользы.

In [347]:
df.drop(index=wrong_list, axis=1, inplace=True)

In [348]:
df.shape[0]

4276

Строки успешно удалены.

В датасете есть много данных о зарплате указанных в следующем виде: 'до 70 000 руб. на руки'.
При таком указании заработной платы нам известно только максимальное вознаграждение, а на месте минимального установлен 0, что является некорректным.
Попробуем сгладить эти неточности и восполнить нули.
Ноль можно было бы заменить на значение МРОТ, но у нас в регионах присутствуют города из разных государств, поэтому такой вариант не подходит.
Восполним нули в 2 шага.

**ШАГ 1:**
  - Отфильтровываем датасет по региону и записываем в переменную `filtered_region`;
  - В полученной таблице находим строки, в которых указана минимальная и максимальная зарплата и записываем их в переменную `filtered_salary`;
  - Также в датасете `filtered_region` ищем строки, в которых указаны только максимальная зарплата, а минимальная равна нулю;
  - Берем строку, где есть данные о максимальной зарплате, а минимальная нулевая. Далее фильтруем датасет по значению найд енной максимальной з/п. Полученный на выходе датасет используем для расчета медианной минимальной зарплаты.
  - Ноль заменяем на полученное значение медианы. Таким образом заполняем все возможные нули, а там где это не возможно возвращаем в ячейку прежнее значение. Проделываем эту операцию в цикле.

**ШАГ 2:**
  - К этому шагу переходим для восполнения оставшихся нулей. Так как у нас сильный невосполнимый дефицит данных, то есть два пути: либо мы удаляем эти строки (если процент таких данных от общего количества данных будет 1-3% и не более), либо мы можем воспользоваться методом Ферми и, например, заменить нули половиной суммы максимально указанной работодателем зарплаты. В любом случае искажения данных неизбежны, но эти действия помогут сделать их не столь существенными.
  


In [349]:
# Оценим количество строк, где нет минимальной з/п, но есть максимальная
wrong_row = df[(df['salary_min']==0)&(df['salary_max']!=0)].shape[0]
print(f'Количество строк = {wrong_row}, что составляет {wrong_row*100/len(df):.2f}% от объема данных всего датасета.')

Количество строк = 567, что составляет 13.26% от объема данных всего датасета.


В датасете имеются города, записанные с дополнительным словом 'город'. Пример ниже. Это может помешать фильтрации данных по региону.

In [350]:
df[df['region']=='город Лимасол'].head(3)

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
12,Python Developer (Crypto),3–6 лет,от 5 000 EUR на руки,город Лимасол,0,5000,3,6
92,"Senior Full-stack Developer (to Cyprus, Armenia)",3–6 лет,от 5 000 EUR на руки,город Лимасол,0,5000,3,6
132,"Senior QA Engineer (Automation, Python) to Cyp...",3–6 лет,от 4 500 EUR на руки,город Лимасол,0,4500,3,6


In [351]:
# Перестрахуемся и уберем приставку 'город' там, где это возможно
def city_remove(row):
    region = row['region']
    if 'город' in region:
        region = region.replace('город ', '')
        return region
    else:
        return region
        
df['region'] = df.apply(city_remove, axis=1)

Теперь можно перейти к шагу 1 в восполнении данных с нулями.

In [352]:
# Шаг 1
region_list = df['region'].unique()
for region in region_list:
    # отфильтровываем датасет по региону
    filtered_region = df[df['region'] == region]
    index_f_r = filtered_region.index
    # Из датасета, отфильтрованного по региону, отфильтровываем строки с мин. з/п != 0 и макс. з/п != 0
    filtered_salary = filtered_region[(filtered_region['salary_min'] != 0) &
                                    (filtered_region['salary_max'] != 0)]
    
    for i in index_f_r:
        similar = filtered_salary[filtered_salary['salary_max'] == filtered_region['salary_max'][i]]
        if (filtered_region['salary_min'][i] == 0 and filtered_region['salary_max'][i] != 0 
            and len(filtered_salary) != 1 and len(filtered_salary) != 0):
            if len(similar) > 1:
                s_median = similar['salary_min'].median()
                df['salary_min'][i] = s_median
            else:
                df['salary_min'][i] = df.loc[i, 'salary_min']
        elif (filtered_region['salary_min'][i] == 0 and filtered_region['salary_max'][i] != 0) and (len(filtered_salary) == 1 or len(filtered_salary) == 0):
            df['salary_min'][i] = df.loc[i, 'salary_min']

df['salary_min'] = df['salary_min'].astype('int')

In [353]:
# Оценим количество строк, где нет минимальной з/п, но есть максимальная
wrong_row = df[(df['salary_min']==0)&(df['salary_max']!=0)].shape[0]
print(f'Количество строк = {wrong_row}, что составляет {wrong_row*100/len(df):.2f}% от объема данных всего датасета.')

Количество строк = 334, что составляет 7.81% от объема данных всего датасета.


Удалось восполнить 231 строку. Переходим к шагу 2.

In [354]:
def del_zero(row):
    s_min = row['salary_min']
    s_max = row['salary_max']
    if s_min == 0 and s_max != 0:
        fixed_salary = int(s_max / 2)
        return fixed_salary
    else:
        return s_min
    

In [355]:
df['salary_min'] = df.apply(del_zero, axis=1)

In [356]:
df

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана,200000,1500000,1,3
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,Астана,4500,5000,3,6
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,Нур-Султан (Астана),150000,200000,0,0
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,Алматы,2000,4000,3,6
4,Backend разработчик,1–3 года,з/п не указана,Астана,0,0,1,3
...,...,...,...,...,...,...,...,...
4279,Специалист службы поддержки с техническими зна...,не требуется,от 15 000 до 40 000 руб. на руки,Ижевск,15000,40000,0,0
4280,Web-разработчик junior,не требуется,до 70 000 руб. на руки,Санкт-Петербург,35000,70000,0,0
4281,Web-разработчик,не требуется,з/п не указана,Нижний Новгород,0,0,0,0
4282,Младший аналитик (IT),не требуется,от 100 000 до 130 000 руб. на руки,Москва,100000,130000,0,0


In [357]:
# Снова оценим количество строк, где нет минимальной з/п, но есть максимальная
wrong_row = df[(df['salary_min']==0)&(df['salary_max']!=0)].shape[0]
print(f'Количество строк = {wrong_row}, что составляет {wrong_row*100/len(df):.2f}% от объема данных всего датасета.')

Количество строк = 0, что составляет 0.00% от объема данных всего датасета.


Все возможные нули заменены.  
Так как почти все зарплаты и опыт работы указаны в виде диапазонов, то для более менее корректной оценки введем два дополнительных столбца со средним значением из этих диапазонов. 

In [358]:
df['salary_mean'] = (df['salary_min'] + df['salary_max']) / 2
df['exp_mean'] = (df['exp_min'] + df['exp_max']) / 2
df['salary_mean'] = df['salary_mean'].apply(lambda x: int(x + 0.5))
df['exp_mean'] = df['exp_mean'].apply(lambda x: int(x + 0.5))

In [359]:
df

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max,salary_mean,exp_mean
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана,200000,1500000,1,3,850000,2
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,Астана,4500,5000,3,6,4750,5
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,Нур-Султан (Астана),150000,200000,0,0,175000,0
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,Алматы,2000,4000,3,6,3000,5
4,Backend разработчик,1–3 года,з/п не указана,Астана,0,0,1,3,0,2
...,...,...,...,...,...,...,...,...,...,...
4279,Специалист службы поддержки с техническими зна...,не требуется,от 15 000 до 40 000 руб. на руки,Ижевск,15000,40000,0,0,27500,0
4280,Web-разработчик junior,не требуется,до 70 000 руб. на руки,Санкт-Петербург,35000,70000,0,0,52500,0
4281,Web-разработчик,не требуется,з/п не указана,Нижний Новгород,0,0,0,0,0,0
4282,Младший аналитик (IT),не требуется,от 100 000 до 130 000 руб. на руки,Москва,100000,130000,0,0,115000,0


## Группировка по типам вакансий

Добавим столбец `vacancy_type`, в который позже запишем типы вакансий.  
Группировку вакансий будем делать по этим типам.  
Разделение по тимам будет приблизительным, так как среди наименований вакансий есть те, которые могут относиться сразу к нескольким категориям и типам. Также названия вакансий составлены из русских и английских слов, поэтому провести лемматизацию слов автоматизировано затруднительно.

In [360]:
# Добавляем столбец
df.insert(1, 'vacancy_type', 0)

Разделим вакансии на следующие группы:

№ п/п | Направление | Код
---|---|---
1 | Data Science | DS
2 | Искусственный интеллект | AI
3 | Инженер данных | DE
4 | Большие данные | BD
5 | Глубокое обучение | DL
6 | Анализ данных | DA
7 | Машинное обучение | ML
8 | Software Engenering | SE
9 | Web-разработка | WB
9 | Прочее | other




Составим списки ключевых слов для поиска вакансий.

In [361]:
keywords_dict = {
    'ds_list': ['data science', 'Data Science', 'data Science', 
           'Data-Science', 'data-science', 'Data-science', 
           'data-Science', 'Data scientist', 'data scientist', 
           'data Scientist', 'Data Scientist', 'Data-scientist', 
           'data-scientist', 'data-Scientist', 'Data-Scientist'],

    'ai_list': ['ИИ', 'AI', 'искусственный интеллект', 
           'искусственного интеллекта', 'Искусственный Интеллект', 'Искусственный интеллект',
           'искусственный Интеллект', 'Искусственного Интеллекта', 'Искусственного интеллекта',
           'искусственного Интеллекта', 'artificial intelligence', 'Artificial intelligence', 
           'artificial Intelligence', 'Artificial Intelligence'],

    'de_list': ['Data engeener', 'data engeener', 'Data Engeener', 
            'data Engeener', 'Data engeenering', 'data engeenering', 
            'Data Engeenering', 'data Engeenering', 'Инженер данных', 
            'инженер данных', 'инженер Данных', 'Инженер Данных',
            'Инженер больших данных', 'инженер больших данных', 'инженер Больших Данных', 
            'Инженер Больших Данных', 'Инженер Больших данных', 'инженер Больших данных', 
            'инженер больших Данных', 'Инженер больших Данных', 'дата-инженер', 
            'Дата-инженер', 'Дата-Инженер', 'Data-engeener', 
            'data-engeener', 'Data-Engeener', 'data-Engeener'],

    'bd_list': ['Big Data', 'big data', 'Big data', 
           'big Data', 'Big-Data', 'big-data', 
           'Big-data', 'big-Data', 'Большие Данные', 
           'большие данные', 'Большие данные', 'большие Данные',
           'Больших Данных', 'больших данных', 'Больших данных', 
           'больших Данных'],

    'dl_list': ['DL', 'deep learning', 'deep Learning', 
           'Deep Learning', 'Deep learning', 'deep-learning', 
           'deep-Learning', 'Deep-Learning', 'Deep-learning', 
           'Глубокое Обучение', 'глубокое обучение', 'Глубокое обучение',
           'глубокое Обучение', 'Глубокому Обучению', 'глубокому обучению',
           'Глубокому обучению', 'глубокому Обучению', 'Computer Vision', 
           'computer Vision', 'Computer vision', 'computer vision'],

    'da_list': ['Data Analysis', 'data analysis', 'Data analysis',
           'data Analysis', 'Data-Analysis', 'data-analysis', 
           'Data-analysis', 'data-Analysis', 'Data Analytics',
           'Data analytics', 'data Analytics', 'data analytics',
           'Data-Analytics', 'Data-analytics', 'data-Analytics', 
           'data-analytics', 'Data Analyst', 'data analyst',
           'Data analyst', 'data Analyst', 'Data-Analyst', 
           'data-analyst', 'Data-analyst', 'data-Analyst',
           'Аналитик Данных', 'Аналитик данных', 'аналитик Данных',
           'аналитик данных', 'Data Аналитик', 'data Аналитик', 
           'Data аналитик', 'data аналитик', 'Data-Аналитик', 
           'data-Аналитик', 'Data-аналитик', 'data-аналитик', 
           'Анализ данных', 'анализ Данных', 'анализ данных', 
           'Анализ Данных', 'Data анализ', 'data анализ', 
           'Data Анализ', 'data Анализ', 'Data-анализ', 
           'data-анализ', 'Data-Анализ', 'data-Анализ', 
           'Аналитик', 'аналитик'],

    'ml_list': ['Machine Learning', 'machine Learning', 'Machine learning', 
           'machine learning', 'Machine-Learning', 'machine-Learning', 
           'Machine-learning', 'machine-learning', 'Машинное обучение',
           'машинное обучение', 'Машинное Обучение', 'машинное Обучение',
           'Машинного обучения', 'Машинного Обучения', 'машинного обучения', 
           'машинного Обучения'],

    'se_list': ['Software Engeener', 'software Engeener', 'Software engeener', 
           'software engeener', 'Software Engeenering', 'software Engeenering', 
           'Software engeenering', 'software engeenering', 'Software-Engeener', 
           'software-Engeener', 'Software-engeener', 'software-engeener',
           'Software-Engeenering', 'software-Engeenering', 'Software-engeenering', 
           'software-engeenering', 'Software Developer', 'software Developer', 
           'Software developer', 'software developer', 'Software Developing', 
           'software Developing', 'Software developing', 'software developing',
           'Software Develop', 'software Develop', 'Software develop', 
           'software develop', 'Software-Developer', 'software-Developer', 
           'Software-developer', 'software-developer', 'Software-Developing', 
           'software-Developing', 'Software-developing', 'software-developing',
           'Software-Develop', 'software-Develop', 'Software-develop', 
           'software-develop', 'Разработчик', 'Разработчика', 
           'Разработка', 'разработчик', 'разработчика', 
           'разработка', 'Программного обеспечения', 'Программное обеспечение', 
           'программного обеспечения', 'программное обеспечение', 'Software Инженер', 
           'software Инженер', 'Software инженер', 'software инженер', 
           'Software-Инженер', 'software-Инженер', 'Software-инженер', 
           'software-инженер', 'Инженер', 'инженер', 'Developer', 'developer'],
    'wb_list': ['Web', 'web', 'WEB', 
                'Web-разработчик', 'web-разработчик', 'WEB-разработчик', 
                'Web-Разработчик', 'web-Разработчик', 'WEB-Разработчик',
                'Web разработчик', 'web разработчик', 'WEB разработчик',
                'Web Разработчик', 'web Разработчик', 'WEB Разработчик',
                'верстка', 'Верстка', 'верстальщик', 
                'Верстальщик', 'PHP', 'php', 
                'PHP-разработчик', 'php-разработчик', 'PHP-Разработчик', 
                'php-Разработчик', 'JavaScript', 'javascript', 
                'Javascript', 'javaScript', 'JavaScript-Разработчик', 
                'javascript-Разработчик', 'Javascript-Разработчик', 'javaScript-Разработчик',
                'JavaScript Разработчик', 'javascript Разработчик', 'Javascript Разработчик', 
                'javaScript Разработчик', 'JavaScript-разработчик', 'javascript-разработчик', 
                'Javascript-разработчик', 'javaScript-разработчик','JavaScript разработчик', 
                'javascript разработчик', 'Javascript разработчик', 'javaScript разработчик',
                'лендинг', 'Лендинг', 'Сайт', 
                'Сайтов', 'сайт', 'сайтов', 
                'laravel', 'Laravel', 'Django', 
                'django', 'веб-разработчик', 'Веб-разработчик', 
                'ВЕБ-разработчик', 'веб-Разработчик', 'Веб-Разработчик', 
                'ВЕБ-Разработчик', 'веб разработчик', 'Веб разработчик', 
                'ВЕБ разработчик', 'веб Разработчик', 'Веб Разработчик', 
                'ВЕБ Разработчик', 'Pyramid', 'pyramid', 
                'Web2py', 'web2py', 'Flask', 
                'flask', 'bottle', 'Bottle', 
                'Tornado', 'tornado', 'CherryPy', 
                'cherrypy', 'cherryPy', 'Cherrypy', 
                'CodeIgniter', 'codeIgniter', 'Codeigniter', 
                'codeigniter']}

In [362]:
# Посчитаем количество уникальных вакансий
title_unique = df['title'].unique()
title_unique.shape

(2920,)

In [363]:
df.reset_index(inplace=True, drop=True)

In [364]:
def typing(typing_dict, column, code_column, data):
    for key in typing_dict:
        category = key.replace('_list', '').upper()
        for v in typing_dict[key]:
            for i in range(len(data)):
                if (v in str(column[i])) and (code_column[i] == 0):
                    code_column[i] = category
    for n in range(len(data)):
        if code_column[n] == 0:
            code_column[n] = 'other'

In [365]:
typing(keywords_dict, df['title'], df['vacancy_type'], df)

In [366]:
df[df['vacancy_type'] == "WB"]

,title,vacancy_type,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max,salary_mean,exp_mean
14,Automation QA (Python/ JavaScript),WB,1–3 года,з/п не указана,Астана,0,0,1,3,0,2
193,Middle Web QA Engineer,WB,1–3 года,от 850 000 до 1 150 000 KZT на руки,Алматы,850000,1150000,1,3,1000000,2
196,Senior Web QA,WB,3–6 лет,з/п не указана,Алматы,0,0,3,6,0,5
216,Web-программист,WB,1–3 года,от 400 000 до 400 000 KZT на руки,Алматы,400000,400000,1,3,400000,2
218,PHP-программист,WB,1–3 года,от 350 000 до 450 000 KZT на руки,Алматы,350000,450000,1,3,400000,2
254,Программист фронт JavaScript (Senior/Middle),WB,3–6 лет,от 1 500 USD на руки,Минск,750,1500,3,6,1125,5
257,Программист Python (Django),WB,1–3 года,з/п не указана,Минск,0,0,1,3,0,2
358,Senior Product / Web Analyst,WB,3–6 лет,з/п не указана,Тбилиси,0,0,3,6,0,5
550,Тестировщик web-приложений,WB,1–3 года,з/п не указана,Ташкент,0,0,1,3,0,2
553,Fullstack Web Development Instructor,WB,1–3 года,з/п не указана,Фергана,0,0,1,3,0,2


In [367]:
df['vacancy_type'].value_counts()

SE       2370
other    1467
DA        248
DS         59
WB         49
BD         29
AI         28
ML         12
DL          9
DE          5
Name: vacancy_type, dtype: int64

In [158]:
dicted = {'title': ['Python Разработчик'], 'vacancy_type': [np.nan]}
m =pd.DataFrame(dicted)

In [187]:
pd.isnull(m['vacancy_type'][0])

True

In [172]:
'Разработчик' in str(m['title'][0])

True

In [184]:
('Разработчик' in str(m['title'][0])) and (np.isnan(m['vacancy_type'][0]))

True

In [160]:
m

,title,vacancy_type
0,Python Разработчик,NaN


In [154]:
df

,title,vacancy_type,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max,salary_mean,exp_mean
0,Python Разработчик,NaN,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана,200000,1500000,1,3,850000,2
1,Full-stack Python Developer,NaN,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,Астана,4500,5000,3,6,4750,5
2,Junior разработчик/программист в Python,NaN,не требуется,от 150 000 до 200 000 KZT до вычета налогов,Нур-Султан (Астана),150000,200000,0,0,175000,0
3,Python Developer (Middle/Senior),NaN,3–6 лет,от 2 000 до 4 000 USD на руки,Алматы,2000,4000,3,6,3000,5
4,Backend разработчик,NaN,1–3 года,з/п не указана,Астана,0,0,1,3,0,2
...,...,...,...,...,...,...,...,...,...,...,...
4271,Специалист службы поддержки с техническими зна...,NaN,не требуется,от 15 000 до 40 000 руб. на руки,Ижевск,15000,40000,0,0,27500,0
4272,Web-разработчик junior,NaN,не требуется,до 70 000 руб. на руки,Санкт-Петербург,35000,70000,0,0,52500,0
4273,Web-разработчик,NaN,не требуется,з/п не указана,Нижний Новгород,0,0,0,0,0,0
4274,Младший аналитик (IT),NaN,не требуется,от 100 000 до 130 000 руб. на руки,Москва,100000,130000,0,0,115000,0


# Чек-лист

- [ ]  Сгруппируйте вакансии по направлениями (DS, DE, Software Engenering, etc.);
- [ ]  Какая средняя и медианная зарплата по группам вакансий?
- [ ]  Какая средняя и медианная зарплата по каждому региону?
- [ ]  Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?
- [ ]  Какое процентное соотношение каждого региона по вакансиям от всех вакансий?
- [ ]  Какая корреляция уровня опыта от зарплаты? (Преобразовать уровень опыта в числовой тип, если возможно. Например: ('MI': 0, 'SE': 1, 'EN': 2, 'EX': 3);
- [ ]  Сколько должностей в наборе данных?
- [ ]  Какие 10 наиболее часто встречающихся должностей?


In [ ]:


for v in de_names:
     for i in range(len(title_unique)):
          if v in str(title_unique[i]):
               i_number = df.loc[df['title'] == title_unique[i]].index
               de_title_list.append(i_number[0])
               #de_title_list.append(df.loc[df['title'] == title_unique[i]].index)
print(de_title_list)


In [ ]:
k = df[df['region'] == 'Астана']
index_k = k.index
for index in index_k:
    print(k[index])

In [ ]:
k

In [247]:
df[(df['salary_min']==0)&(df['salary_max']!=0)]

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
12,Python Developer (Crypto),3–6 лет,от 5 000 EUR на руки,город Лимасол,0,5000,3,6
22,Go разработчик со знаниями Python,1–3 года,до 1 000 000 KZT на руки,Алматы,0,1000000,1,3
45,Senior Python разработчик (back-end),более 6 лет,от 1 500 000 KZT до вычета налогов,Нур-Султан (Астана),0,1500000,6,0
65,Senior Go Developer,3–6 лет,до 1 300 000 KZT на руки,Алматы,0,1300000,3,6
77,PHP-разработчик,1–3 года,от 300 000 KZT на руки,Алматы,0,300000,1,3
...,...,...,...,...,...,...,...,...
4265,Ночной специалист службы поддержки с техническ...,не требуется,от 15 000 руб. на руки,Москва,0,15000,0,0
4266,Ночной специалист службы поддержки с техническ...,не требуется,от 15 000 руб. на руки,Санкт-Петербург,0,15000,0,0
4270,Преподаватель программирования для детей,не требуется,до 45 000 руб. на руки,Москва,0,45000,0,0
4271,Специалист по внедрению и сопровождению ПО,не требуется,от 45 000 руб. на руки,Челябинск,0,45000,0,0


In [248]:
567 - 336

231

In [249]:
336*100/4284

7.8431372549019605

4

In [222]:


# отфильтровываем датасет по региону
filtred_region = df[df['region'] == 'Москва']
index_f_r = filtred_region.index
# Из датасета, отфильтрованного по региону, отфильтровываем строки с мин. з/п != 0 и макс. з/п != 0
filtred_salary = filtred_region[(filtred_region['salary_min'] != 0) &
                                (filtred_region['salary_max'] != 0)]

In [ ]:
for i in index_f_r:
    similar = filtred_salary[filtred_salary['salary_max'] == filtred_region['salary_max'][i]]
    if (filtred_region['salary_min'][i] == 0 and filtred_region['salary_max'][i] != 0 
        and len(filtred_salary) != 1 and len(filtred_salary) != 0):
        if len(similar) > 1:
            s_median = similar['salary_min'].median()
            df['salary_min'][i] = s_median
        else:
            df['salary_min'][i] = df.loc[i, 'salary_min']
    elif (filtred_region['salary_min'][i] == 0 and filtred_region['salary_max'][i] != 0) and (len(filtred_salary) == 1 or len(filtred_salary) == 0):
        df['salary_min'][i] = df.loc[i, 'salary_min']

In [231]:
s_median = similar['salary_min'].median()

In [232]:
s_median

180000.0

In [227]:
similar = filtred_salary[filtred_salary['salary_max'] == filtred_region['salary_max'][601]]

In [224]:
filtred_region['salary_max'][601]

230000

In [202]:
s_median

title              NaN
work_experience    NaN
salary             NaN
region             NaN
salary_min         NaN
salary_max         NaN
exp_min            NaN
exp_max            NaN
dtype: object

In [225]:
(filtred_region['salary_min'][601] == 0 and filtred_region['salary_max'][601] != 0 and len(filtred_salary) != 1 and len(filtred_salary) != 0)

True

In [226]:
(filtred_region['salary_min'][601] == 0 and filtred_region['salary_max'][601] != 0) and (len(filtred_salary) == 1 or len(filtred_salary) == 0)

False

In [ ]:
df['salary_min'] = df['salary_min'].astype('int')